In [1]:
import torch.nn   as nn
import GM_library as GML
import numpy      as np
import torch
import json

from os                   import path, listdir
from torch_geometric.data import Data

import sys
sys.path.append('../')
import MP.MP_library as MPL

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Target to generate new crystals
target = 'GM_EPA'

# In case database is created from scratch (otherwise, it is not being used)
DB_path = '../MP/Loaded_EMP'

input_folder    = 'models'
target_folder   = f'{input_folder}/{target}'
edge_model_name = f'{target_folder}/edge_model.pt'
node_model_name = f'{target_folder}/node_model.pt'

In [3]:
# Read the file in JSON format to a dictionary
with open(f'{target_folder}/model_parameters.json', 'r') as json_file:
    model_parameters = json.load(json_file)

# Number of graphs to predict
N_predictions = 10

# Number of diffusing and denoising steps
n_t_steps = model_parameters['n_t_steps']

# Decay of parameter alpha
noise_contribution = model_parameters['noise_contribution']
alpha_decay = 0.5 * (1 - noise_contribution**2)

# Dropouts for node and edge models (independent of each other)
dropout_node = model_parameters['dropout_node']
dropout_edge = model_parameters['dropout_edge']

# Define box shape
L = model_parameters['L']

# Generation of graph database for training

Load the datasets, already standarized if possible.

In [4]:
labels_name         = f'{target_folder}/labels.pt'
dataset_name        = f'{target_folder}/dataset.pt'
dataset_name_std    = f'{target_folder}/standardized_dataset.pt'
parameters_name_std = f'{target_folder}/standardized_parameters.pt'  # Parameters for rescaling the predictions

# Load the standardized dataset, with corresponding labels and parameters
dataset    = torch.load(dataset_name_std)
parameters = torch.load(parameters_name_std)

# Assigning parameters accordingly
target_mean, feat_mean, edge_mean, target_std, edge_std, feat_std, scale = parameters

# Defining target factor
target_factor = target_std / scale

In [5]:
# Calculate the mean and standard deviation of the number of nodes
total_nodes = torch.tensor([data.num_nodes for data in dataset])
mean_nodes  = torch.mean(total_nodes.float()).item()
std_nodes   = torch.std(total_nodes.float()).item()

mean_nodes, std_nodes

(623.8933715820312, 164.2827606201172)

# Loading the model

In [6]:
# Determine number of features in dataset, considering the t_step information
n_features = dataset[0].num_node_features + 1

# Instantiate the models for nodes and edges
node_model = GML.nGCNN(n_features, dropout_node).to(device)
node_model.load_state_dict(torch.load(node_model_name))
node_model.eval()

edge_model = GML.eGCNN(n_features, dropout_edge).to(device)
edge_model.load_state_dict(torch.load(edge_model_name))
edge_model.eval()

print('\nNode GCNN:')
print(node_model)
print('\nEdge GCNN:')
print(edge_model)


Node GCNN:
nGCNN(
  (conv1): GraphConv(5, 256)
  (conv2): GraphConv(256, 256)
  (conv3): GraphConv(256, 5)
)

Edge GCNN:
eGCNN(
  (linear1): Linear(in_features=6, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=1, bias=True)
)


# Generating new cystals

In [7]:
# Predicting loop
predicted_dataset = []
with torch.no_grad():
    for i in range(N_predictions):
        # Get random number of nodes
        n_nodes = int(np.random.normal(mean_nodes, std_nodes))
        
        # Diffuse the graph with some noise
        diffused_graph = GML.get_random_graph(n_nodes, n_features-1)
        
        # Denoise the diffused graph
        #print(f'Denoising...')
        denoised_graph, _ = GML.denoise(diffused_graph, n_t_steps, node_model, edge_model,
                                        s=alpha_decay)
        
        # Append generated graph
        predicted_dataset.append(denoised_graph)

# Revert stardadization
denoised_graphs = GML.revert_standardize_dataset(predicted_dataset, parameters)
denoised_graphs

[Data(x=[933, 4], edge_index=[2, 434778], edge_attr=[434778]),
 Data(x=[819, 4], edge_index=[2, 334971], edge_attr=[334971]),
 Data(x=[230, 4], edge_index=[2, 26335], edge_attr=[26335]),
 Data(x=[856, 4], edge_index=[2, 365940], edge_attr=[365940]),
 Data(x=[756, 4], edge_index=[2, 285390], edge_attr=[285390]),
 Data(x=[668, 4], edge_index=[2, 222778], edge_attr=[222778]),
 Data(x=[695, 4], edge_index=[2, 241165], edge_attr=[241165]),
 Data(x=[510, 4], edge_index=[2, 129795], edge_attr=[129795]),
 Data(x=[709, 4], edge_index=[2, 250986], edge_attr=[250986]),
 Data(x=[718, 4], edge_index=[2, 257403], edge_attr=[257403])]

In [8]:
for i in range(N_predictions):
    graph = denoised_graphs[i].clone()
    try:
        GML.POSCAR_graph_encoding(graph, L, file_name=f'POSCAR-{i}', POSCAR_directory='./')
    except SystemExit:
        continue

Invalid graph, atoms overlapping. Applying brute force :)


/Users/cibran/Work/UPC/GenerativeModels/GM_library.py:1048: RuntimeWarning: invalid value encountered in sqrt
  area = np.sqrt(s * (s - a) * (s - b) * (s - c))


0 27 164 19.450502 26.512428 20.851635 16.61757116818033 20.6582957179033
Invalid graph, atoms overlapping. Applying brute force :)
0 1 534 20.356823 23.072216 21.379894 12.026127957340583 19.69008377509511
Invalid graph, atoms overlapping. Applying brute force :)
0 3 98 17.173834 24.823483 26.81605 5.591219175599476 24.185606302610264
Invalid graph, atoms overlapping. Applying brute force :)
0 7 344 18.238218 23.255348 25.007807 6.800374443609012 22.238842766404236
Invalid graph, atoms overlapping. Applying brute force :)
0 10 99 19.23074 21.17076 26.358501 3.2045515462212286 20.926822370738613
Invalid graph, atoms overlapping. Applying brute force :)
0 1 103 23.807392 25.012999 18.06712 18.188073742424585 17.171024184650793
Invalid graph, atoms overlapping. Applying brute force :)
0 6 405 19.697622 25.511206 23.784534 12.009415470219722 22.507677689287846
Invalid graph, atoms overlapping. Applying brute force :)
0 5 347 19.695868 23.147991 22.204895 10.933748668643197 20.403005556528